In [273]:
import pandas as pd
import numpy as np
pd.set_option('expand_frame_repr', True)
award_string = 'SBAHQ-'

## Find Files and Columns
Using a sample award id string, look through JAAMS and Prism files to see which ones contain award ids.
For files that contain award id data, find out which column(s) the award id is in.

In [274]:
#assuming that award ids for this data subset are start with 'SBAHQ-', 
#find the JAAMS and Prism tables that contain award is somewhere
#in their data
awards_files = !grep --include=\*.txt -rnl -e 'SBAHQ-'  data
jaams_awards_files = [f.split('\n')[0] for f in awards_files if f.find('jaams') > -1]
prism_awards_files = [f.split('\n')[0] for f in awards_files if f.find('prism') > -1]
prism = {}
#raw prism files aren't quoted, so many lines won't parse (skip them for now)
for fi in prism_awards_files:
    key = fi.split('/')[-1][:-4].lower()
    prism[key] = pd.read_csv(fi, error_bad_lines = False, warn_bad_lines = False, encoding="latin_1")
    prism[key].rename(columns=lambda x: '{}.'.format(key) + x.lower(), inplace = True)
prism_csv = pd.read_csv('data/data_act_prism_grants_fy14.csv', encoding="latin_1")
jaams = {}
for fi in jaams_awards_files:
    key = fi.split('/')[-1][:-4].lower()
    jaams[key] = pd.read_csv(fi)
    jaams[key].rename(columns=lambda x: '{}.'.format(key) + x.lower(), inplace = True)

### JAAMS

In [275]:
print('looking for JAAMS colums that have an award id\n')
for k, df in jaams.items():
    for col in df:
        if df[col].to_string().find(award_string) >= 0:
            print ('{}: {}'.format(k, col))

looking for JAAMS colums that have an award id

po_lines_all: po_lines_all.attribute1
ap_invoice_distributions_all: ap_invoice_distributions_all.attribute3
po_requisition_headers_all: po_requisition_headers_all.attribute2
ap_invoices_all: ap_invoices_all.invoice_num
ap_invoices_all: ap_invoices_all.attribute11
po_headers_all: po_headers_all.segment1


### Prism

In [276]:
print('looking for Prism colums that have an award id\n')
for k, df in prism.items():
    for col in df:
        if df[col].to_string().find(award_string) >= 0:
            print ('{}: {}'.format(k, col))
for col in prism_csv:
    if prism_csv[col].to_string().find(award_string) >= 0:
        print ('prism csv: {}'.format(col))

looking for Prism colums that have an award id

grantheader: grantheader.sba1222progcode
grantheader: grantheader.sba1222budgetcode
grantheader: grantheader.sba1222documentno
faadsciv: faadsciv.docnum
header: header.docnum
prism csv: Grant Number
prism csv: Award ID
prism csv: Unique Identifer


## Joining JAAMS and Prism

Very, very preliminary sketch of relationship between Prism and JAAMS.
* According to the SBA mapping doc, the unique award id is in JAAMS: PO_HEADERS_ALL.SEGMENT1
* Obviously we need an award id in Prism to join the two systems. A few Prism tables had columns that contain award id-looking information:
    * header.docnum
    * grantheader.sba1222progcode
    * grantheader.sba1222budgetcode
    * grantheader.sba1222documentno
    * faadsciv.docnum
    
The columns in grantheader look like they are free-text columns that sometimes contain what looks like the JAAMS award id. It doesn't seem like the right match.

![Prism/JAAMS relationship](prism_jaams_relationship.png 'Prism/JAAMS relatinoship')

## JAAMS PO_HEADERS_ALL to Prism Header.docnum

This matches 18 records between JAAMS and Prism (a single award id: SBAHQ-14-S-0001)

In [277]:
header = prism['header']
po_headers_all = jaams['po_headers_all']
po_headers_all['po_headers_all.segment1_stripped'] = po_headers_all['po_headers_all.segment1'].replace('-', '', regex = True)
#remove some strange rows from the header file
header = header.dropna(subset = ['header.docnum', 'header.dockey'])
header['header.docnum_stripped'] = header['header.docnum'].replace('-', '', regex = True)

jp_merge = pd.merge(
    po_headers_all,
    header,
    left_on = 'po_headers_all.segment1_stripped',
    right_on = 'header.docnum_stripped')
print ('Number of po_headers_all records matched to prism header: {}'.format(len(jp_merge.index)))
print ('Award ids matched: {}'.format(pd.unique(jp_merge['po_headers_all.segment1'].ravel())))

Number of po_headers_all records matched to prism header: 18
Award ids matched: ['SBAHQ-14-S-0001']


/Users/rebeccasweger/Dev/.virtualenvs/intercessor/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## JAAMS PO_HEADERS_ALL to Prism faadsciv

Nothing matched, even though it looks like it should (i.e., SBAHQ-14-S-0001 is in the faadsciv file.) Deferring this for now, since it doesn't make sense to match to something faads (i.e., grants) specific)

In [278]:
#faadsciv = prism['faadsciv']
#faadsciv['faadsciv.docnum_stripped'] = faadsciv['faadsciv.docnum'].replace('-', '', regex = True)
#jp_merge = pd.merge(
#    po_headers_all,
#    faadsciv,
#    left_on = 'po_headers_all.segment1_stripped',
#    right_on = 'faadsciv.docnum_stripped')
#print ('Number of po_headers_all records matched to prism faadsciv: {}'.format(len(jp_merge.index)))
#print ('Award ids matched: {}'.format(pd.unique(jp_merge['po_headers_all.segment1'].ravel())))

## Merge in Other Required Data

### Vendor
According to mapping doc, the first address line of the awardee/recipient comes from SAM. The remaining portions of the address are mapped to Prism table docvendor.

In [279]:
jp_merge['header.dockey'] = jp_merge['header.dockey'].astype(np.int64)
jp_merge['header.verkey'] = jp_merge['header.verkey'].astype(np.int64)
jp_merge.to_csv('jpmerge_before.csv', index = False)
docvendor = pd.read_csv(
    'data/prism/docVendor.txt',
    index_col = False,
    error_bad_lines = False,
    warn_bad_lines = False
)
docvendor.rename(columns = lambda x: 'docvendor.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, docvendor,
    left_on = ['header.dockey', 'header.verkey'],
    right_on = ['docvendor.dockey', 'docvendor.verkey']
)

### PO_LINES_ALL (for award amount info)

This join was fairly straighforward, since it was included in the sample SQL we got)

In [280]:
po_lines_all = jaams['po_lines_all']
jp_merge = pd.merge(
    jp_merge, po_lines_all,
    left_on = ['po_headers_all.po_header_id'],
    right_on = ['po_lines_all.po_header_id']
)
len(jp_merge.index)

162

In [281]:
jp_merge.to_csv('jp_merge.csv', index = False)
len(jp_merge.index)

162

### PO_DISTRIBUTIONS_ALL (for type of transaction)

In [282]:
po_distributions_all = pd.read_csv('data/jaams/PO_DISTRIBUTIONS_ALL.txt')
po_distributions_all.rename(columns = lambda x: 'po_distributions_all.' + x.lower(), inplace = True)
jp_merge = pd.merge(
    jp_merge, po_distributions_all,
    left_on = ['po_lines_all.po_header_id', 'po_lines_all.po_line_id'],
    right_on = ['po_distributions_all.po_header_id', 'po_distributions_all.po_line_id']
)

## Add Calculated Fields

In [283]:
jp_merge['po_lines_all.total_amount'] = jp_merge['po_lines_all.quantity'] * jp_merge['po_lines_all.unit_price']
jp_merge['funding_agency_name'] = 'Small Business Administration'
jp_merge['funding_agency_code'] = 'fill in SBA code here'
jp_merge['funding_sub_tier_agency_name'] = 'Small Business Administration'
jp_merge['funding_sub_tier_agency_code'] = 'fill in SBA code here'

## Reduce the Huge Merged File to DATA Act Elements

In [290]:
data_act = jp_merge[[
    'po_lines_all.item_description', #award description
    'po_headers_all.segment1', #award id
    'header.issuingdocaddresskey', #awarding office code
    'header.awarddate', #action date
    'docvendor.name', #Recipient name
    'po_distributions_all.attribute10', #period of performance start date
    'po_distributions_all.attribute11', #period of performance end date
    'docvendor.duns', #
    'docvendor.dunsplus4',
    'docvendor.address1',
    'po_lines_all.total_amount', #funding action obligation (does not account for cancellations etc.)
    'funding_agency_name',
    'funding_agency_code',
    'funding_sub_tier_agency_name',
    'funding_sub_tier_agency_code',
    'po_distributions_all.attribute_category' #type of award code
    ]]

In [291]:
data_act.to_csv('data/data_act.csv', index = False)